# Bayesian Personalized Ranking (BPR)

## 0 Global Settings and Imports

In [1]:
import sys
import os
import cornac
import pandas as pd
import numpy as np
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.utils.constants import (
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_RATING_COL,
    DEFAULT_PREDICTION_COL,
    DEFAULT_RELEVANCE_COL,
    DEFAULT_SIMILARITY_COL,
    DEFAULT_ITEM_FEATURES_COL,
    DEFAULT_ITEM_SIM_MEASURE,
    DEFAULT_K,
    DEFAULT_THRESHOLD,
)
DEFAULT_RNDCG_MULTIPLIER = 3

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

/Users/DanielShen/opt/anaconda3/envs/microsoft_rec/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.
System version: 3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:51:59) 
[Clang 14.0.6 ]
Cornac version: 1.15.4


In [2]:
THIS_ENGINE_NAME = "bpr_field_8"

# DATA_FILE_NAME = "../Data/20230813T210908_sales_6mo_basic_single_events_removed.csv"
# DATA_FILE_NAME = "../Data/20230811T031507_sales_12mo_basic_single_events_removed.csv"
DATA_FILE_NAME = "../Data/20230809T003134_sales_25mo_basic_single_events_removed.csv"
COL_USER = "location_id"
COL_ITEM = "product"
COL_RATING = "sold_count"

# country
COUNTRY = "nigeria"

# top k items to recommend, for train & test
TOP_K_SPLIT_TRAIN_TEST = 10
# top k items to recommend, for final product recommendation output
TOP_K_WHOLE = 10
SAVE_ALL_RECS = False
SAVE_NEW_RECS = False

# fraction of location_skus to include in training dataset
TRAIN_FRAC = 0.75

# top MULTIPLIER * k items are considered relevant for nDCG
RNDCG_MULTIPLIER = 3

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100
LEARNING_RATE = 0.01
LAMBDA_REG = 0.001

### Revised/copied code from packages

In [3]:
def _get_rating_column(relevancy_method: str, **kwargs) -> str:
    r"""Helper utility to simplify the arguments of eval metrics
    Attemtps to address https://github.com/microsoft/recommenders/issues/1737.

    Args:
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.

    Returns:
        str: rating column name.
    """
    if relevancy_method != "top_k":
        if "col_rating" not in kwargs:
            raise ValueError("Expected an argument `col_rating` but wasn't found.")
        col_rating = kwargs.get("col_rating")
    else:
        col_rating = kwargs.get("col_rating", DEFAULT_RATING_COL)
    return col_rating


def get_top_k_items(
    dataframe, col_user=DEFAULT_USER_COL, col_rating=DEFAULT_RATING_COL, k=DEFAULT_K
):
    """Get the input customer-item-rating tuple in the format of Pandas
    DataFrame, output a Pandas DataFrame in the dense format of top k items
    for each user.

    Note:
        If it is implicit rating, just append a column of constants to be
        ratings.

    Args:
        dataframe (pandas.DataFrame): DataFrame of rating data (in the format
        customerID-itemID-rating)
        col_user (str): column name for user
        col_rating (str): column name for rating
        k (int or None): number of items for each user; None means that the input has already been
        filtered out top k items and sorted by ratings and there is no need to do that again.

    Returns:
        pandas.DataFrame: DataFrame of top k items for each user, sorted by `col_user` and `rank`
    """
    # Sort dataframe by col_user and (top k) col_rating
    if k is None:
        top_k_items = dataframe
    else:
        top_k_items = (
            dataframe.sort_values([col_user, col_rating], ascending=[True, False])
            .groupby(col_user, as_index=False)
            .head(k)
            .reset_index(drop=True)
        )
    # Add ranks
    top_k_items["rank"] = top_k_items.groupby(col_user, sort=False).cumcount() + 1
    return top_k_items


def merge_ranking_true_pred_new(
    rating_true,
    rating_pred,
    col_user,
    col_item,
    col_rating,
    col_prediction,
    relevancy_method,
    k=DEFAULT_K,
    rndcg_multiplier=DEFAULT_RNDCG_MULTIPLIER,
    threshold=DEFAULT_THRESHOLD,
):
    """Filter truth and prediction data frames on common users

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user (optional)
        threshold (float): threshold of top items per user (optional)

    Returns:
        pandas.DataFrame: DataFrame of recommendation hits, sorted by `col_user` and `rank`
    """

    # Make sure the prediction and true data frames have the same set of users
    common_users = set(rating_true[col_user]).intersection(set(rating_pred[col_user]))
    rating_true_common = rating_true[rating_true[col_user].isin(common_users)]
    rating_pred_common = rating_pred[rating_pred[col_user].isin(common_users)]

    # Return hit items in prediction data frame with ranking information. This is used for calculating NDCG and MAP.
    # Use first to generate unique ranking values for each item. This is to align with the implementation in
    # Spark evaluation metrics, where index of each recommended items (the indices are unique to items) is used
    # to calculate penalized precision of the ordered items.
    if relevancy_method == "top_k":
        top_k = k
    elif relevancy_method == "by_threshold":
        top_k = threshold
    elif relevancy_method is None:
        top_k = None
    else:
        raise NotImplementedError("Invalid relevancy_method")
    df_hit = get_top_k_items(
        dataframe=rating_pred_common,
        col_user=col_user,
        col_rating=col_prediction,
        k=top_k,
    )
    rating_true_common_top_mult_k = get_top_k_items(
        dataframe=rating_true_common,
        col_user=col_user,
        col_rating=col_rating,
        k=RNDCG_MULTIPLIER * top_k,
    )[[col_user, col_item]]
    df_hit = pd.merge(df_hit, rating_true_common_top_mult_k, on=[col_user, col_item])[
        [col_user, col_item, "rank"]
    ]

    return df_hit


def rndcg_at_k(
    rating_true,
    rating_pred,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_prediction=DEFAULT_PREDICTION_COL,
    relevancy_method="top_k",
    k=DEFAULT_K,
    rndcg_multiplier=RNDCG_MULTIPLIER,
    threshold=DEFAULT_THRESHOLD,
    score_type="binary",
    discfun_type="loge",
    **kwargs
):
    """Normalized Discounted Cumulative Gain (nDCG).

    Info: https://en.wikipedia.org/wiki/Discounted_cumulative_gain

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user
        threshold (float): threshold of top items per user (optional)
        score_type (str): type of relevance scores ['binary', 'raw', 'exp']. With the default option 'binary', the
            relevance score is reduced to either 1 (hit) or 0 (miss). Option 'raw' uses the raw relevance score.
            Option 'exp' uses (2 ** RAW_RELEVANCE - 1) as the relevance score
        discfun_type (str): type of discount function ['loge', 'log2'] used to calculate DCG.

    Returns:
        float: nDCG at k (min=0, max=1).
    """
    col_rating = _get_rating_column(relevancy_method, **kwargs)
    df_hit = merge_ranking_true_pred_new(
        rating_true=rating_true,
        rating_pred=rating_pred,
        col_user=col_user,
        col_item=col_item,
        col_rating=col_rating,
        col_prediction=col_prediction,
        relevancy_method=relevancy_method,
        k=k,
        rndcg_multiplier=rndcg_multiplier,
        threshold=threshold,
    )

    if df_hit.shape[0] == 0:
        return 0.0

    df_dcg = df_hit.merge(rating_pred, on=[col_user, col_item]).merge(
        rating_true, on=[col_user, col_item], how="outer", suffixes=("_left", None)
    )

    if score_type == "binary":
        df_dcg["rel"] = 1
    elif score_type == "raw":
        df_dcg["rel"] = df_dcg[col_rating]
    elif score_type == "exp":
        df_dcg["rel"] = 2 ** df_dcg[col_rating] - 1
    else:
        raise ValueError("score_type must be one of 'binary', 'raw', 'exp'")

    if discfun_type == "loge":
        discfun = np.log
    elif discfun_type == "log2":
        discfun = np.log2
    else:
        raise ValueError("discfun_type must be one of 'loge', 'log2'")

    # Calculate the actual discounted gain for each record
    df_dcg["dcg"] = df_dcg["rel"] / discfun(1 + df_dcg["rank"])

    # Calculate the ideal discounted gain for each record
    df_idcg = df_dcg.sort_values([col_user, col_rating], ascending=False)
    df_idcg["irank"] = df_idcg.groupby(col_user, as_index=False, sort=False)[
        col_rating
    ].rank("first", ascending=False)
    df_idcg["idcg"] = df_idcg["rel"] / discfun(1 + df_idcg["irank"])

    # Calculate the actual DCG for each user
    df_user = df_dcg.groupby(col_user, as_index=False, sort=False).agg({"dcg": "sum"})

    # Calculate the ideal DCG for each user
    df_user = df_user.merge(
        df_idcg.groupby(col_user, as_index=False, sort=False)
        .head(k)
        .groupby(col_user, as_index=False, sort=False)
        .agg({"idcg": "sum"}),
        on=col_user,
    )

    # DCG over IDCG is the normalized DCG
    df_user["ndcg"] = df_user["dcg"] / df_user["idcg"]
    return df_user["ndcg"].mean()

## 1 Data


### 1.1 Load and split data

To evaluate the performance of item recommendation, we adopted the provided `python_random_split` tool for the consistency.  Data is randomly split into training and test sets with the ratio of 75/25.


Note that Cornac also cover different [built-in schemes](https://cornac.readthedocs.io/en/latest/eval_methods.html) for model evaluation.

In [4]:
data_all_cols = pd.read_csv(DATA_FILE_NAME)
data_all_cols = data_all_cols[data_all_cols["country"] == COUNTRY]

data = data_all_cols[[COL_USER, COL_ITEM, COL_RATING]]

data.head()

,location_id,product,sold_count
0,"""01cdce0f-96cb-4706-8eb5-ca1c3e52cee2""",Coartem 80/480mg Tablets x6,9
1,"""02c51d76-3932-45d6-84c8-e40917ceda22""",Coartem 80/480mg Tablets x6,12
2,"""03550e79-e8f4-475c-be2b-ea978a197f29""",Coartem 80/480mg Tablets x6,8
3,"""04f7dedc-48c3-4568-b758-6e877e6aec15""",Coartem 80/480mg Tablets x6,198
4,"""05b94f69-344f-4db7-ae3b-1c02cb2d3bdc""",Coartem 80/480mg Tablets x6,339


Here, we use quantity sold, not revenue of sales, for each product.

In [5]:
train, test = python_random_split(data, TRAIN_FRAC, seed = SEED)

### 1.2 Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.

Dataset Class in Cornac serves as the main object that the models will interact with.  In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [6]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 960
Number of items: 2072


/Users/DanielShen/opt/anaconda3/envs/microsoft_rec/lib/python3.8/site-packages/cornac/data/dataset.py:361: UserWarning: 311 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


## 2 Training

The BPR has a few important parameters that we need to consider:

- `k`: controls the dimension of the latent space (i.e. the size of the vectors  $w_u$  and  $h_i$ ).
- `max_iter`: defines the number of iterations of the SGD procedure.
- `learning_rate`: controls the step size $\alpha$ in the gradient update rules.
- `lambda_reg`: controls the L2-Regularization $\lambda$ in the objective function.

Note that different values of `k` and `max_iter` will affect the training time.

We will here set `k` to 200, `max_iter` to 100, `learning_rate` to 0.01, and `lambda_reg` to 0.001. To train the model, we simply need to call the `fit()` method.

In [7]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    lambda_reg=LAMBDA_REG,
    verbose=True,
    seed=SEED
)

In [8]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 100/100 [00:03<00:00, 30.64it/s, correct=95.18%, skipped=11.17%]

Optimization finished!
Took 3.2991 seconds for training.


## 3 Prediction

Now that our model is trained, we can produce the ranked lists for recommendation.  Every recommender models in Cornac provide `rate()` and `rank()` methods for predicting item rated value as well as item ranked list for a given user.  To make use of the current evaluation schemes, we will through `predict()` and `predict_ranking()` functions inside `cornac_utils` to produce the predictions.

Note that BPR model is effectively designed for item ranking.  Hence, we only measure the performance using ranking metrics.

In [9]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol=COL_USER, itemcol=COL_ITEM, remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 1.2418 seconds for prediction.


In [10]:
all_predictions.head()

,location_id,product,prediction
151460,"""f6587c0b-ee2a-4bee-bbbd-d4d7f577e7ea""",Buscomac Tab 10mg,-0.678197
151461,"""f6587c0b-ee2a-4bee-bbbd-d4d7f577e7ea""",Dicnac Diclofenac 50mg Tablets x10,-3.891816
151462,"""f6587c0b-ee2a-4bee-bbbd-d4d7f577e7ea""",Folic acid + B12 Tablets (Dr Meyer's) x100,3.941303
151463,"""f6587c0b-ee2a-4bee-bbbd-d4d7f577e7ea""",Benzyl benzoate 100ml,-0.761906
151464,"""f6587c0b-ee2a-4bee-bbbd-d4d7f577e7ea""",Ulgicid 200ml Suspension,-0.067324


In [11]:
# Explore min
all_predictions.sort_values(by="prediction", ascending=True).head()

,location_id,product,prediction
650764,"""04f7dedc-48c3-4568-b758-6e877e6aec15""",Super Apeti 2 x 10 Tablets,-5.812007
542149,"""a8962afb-c3ba-452c-ab86-151ec6bef88f""",Nivea Lotion Body Natural Fairness 250ml,-5.705727
163007,"""b716103f-75f5-489d-a5cf-e1bce75287e6""",Nivea Lotion Body Natural Fairness 250ml,-5.493479
533019,"""2f72781d-28d9-4bd0-867e-d8a0099fa803""",Orelox 40mg/5ml Suspension 100ml,-5.484428
663942,"""df8325ba-a6a4-4129-a7e2-1c083905b76a""",Flagentyl 500mg Tablets x4,-5.457555


In [12]:
# Explore max
all_predictions.sort_values(by="prediction", ascending=False).head()

,location_id,product,prediction
1344806,"""1319c26d-58dc-41a2-9d2c-f93bab659048""",Orheptal Tonic 300ml,7.351610
939274,"""11e29a4a-081d-4c6a-aa7b-9ad4bf6b794c""",Orheptal Tonic 100ml,7.269585
389646,"""80137059-7695-40c8-ae98-4bab39ccf2d4""",Lofnac Tablets x10,7.204725
1182435,"""be680c27-8202-46d3-9719-19500a52206b""",Cotton wool 100g,7.183845
753993,"""0ce6b16d-0e5a-4cd6-9ccc-70d81037c72a""",Orheptal Tonic 300ml,7.168321


## 4 Evaluation / Testing

In [13]:
# eval_map = map_at_k(test, all_predictions, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_SPLIT_TRAIN_TEST)
# eval_ndcg = ndcg_at_k(test, all_predictions, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_SPLIT_TRAIN_TEST)
# eval_rndcg = rndcg_at_k(test, all_predictions, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_SPLIT_TRAIN_TEST, rndcg_multiplier=RNDCG_MULTIPLIER)
# eval_precision = precision_at_k(test, all_predictions, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_SPLIT_TRAIN_TEST)
# eval_recall = recall_at_k(test, all_predictions, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_SPLIT_TRAIN_TEST)

# print("MAP:\t%f" % eval_map,
#       "NDCG:\t%f" % eval_ndcg,
#       "RNDCG:\t%f" % eval_rndcg,
#       "Precision@K:\t%f" % eval_precision,
#       "Recall@K:\t%f" % eval_recall, sep='\n')

## 5 Train, Predict, and Evaluate on Whole Dataset

Earlier, we had split train (for model training) and test (for evaluation). In implementation, we have train = whole dataset, and we can evaluate on test = whole dataset.

In [14]:
## Data
data_set = cornac.data.Dataset.from_uir(data.itertuples(index=False), seed=SEED)
print('Number of users: {}'.format(data_set.num_users))
print('Number of items: {}'.format(data_set.num_items))

## Train
bpr_whole = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    lambda_reg=LAMBDA_REG,
    verbose=True,
    seed=SEED
)
with Timer() as t:
    bpr_whole.fit(data_set)
print("Took {} seconds for training.".format(t))

## Predict
with Timer() as t:
    all_predictions_whole = predict_ranking(bpr_whole, data, usercol=COL_USER, itemcol=COL_ITEM, remove_seen=False)
print("Took {} seconds for prediction.".format(t))

/Users/DanielShen/opt/anaconda3/envs/microsoft_rec/lib/python3.8/site-packages/cornac/data/dataset.py:361: UserWarning: 570 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


Number of users: 960
Number of items: 2099


100%|██████████| 100/100 [00:04<00:00, 21.14it/s, correct=97.10%, skipped=14.53%]


Optimization finished!
Took 4.7947 seconds for training.
Took 0.4430 seconds for prediction.


In [15]:
# Explore min
all_predictions_whole.sort_values(by="prediction", ascending=True).head()

,location_id,product,prediction
7952,"""04f7dedc-48c3-4568-b758-6e877e6aec15""",Super Apeti 2 x 10 Tablets,-7.389133
7871,"""04f7dedc-48c3-4568-b758-6e877e6aec15""",Piscof Syrup,-7.049437
7849,"""04f7dedc-48c3-4568-b758-6e877e6aec15""",Maxiquine Tablets x 10,-6.610123
862245,"""2df94bfb-4f8b-411c-9411-99c4889193c7""",Super Apeti 2 x 10 Tablets,-6.463717
1333276,"""2f72781d-28d9-4bd0-867e-d8a0099fa803""",Orelox 40mg/5ml Suspension 100ml,-6.439687


In [16]:
# Explore max
all_predictions_whole.sort_values(by="prediction", ascending=False).head()

,location_id,product,prediction
1431695,"""c892df4f-ae29-4b8c-85cf-e7914c7a3f0e""",Ampiclox (Unbranded) 500mg Capsules x100,9.714970
860792,"""2df94bfb-4f8b-411c-9411-99c4889193c7""",Amoxil (Beecham) 500mg Capsules,9.567314
566924,"""8b948640-84fe-4b5b-98e6-18d5c1cee21a""",Bunto Blood Caps x100,9.509598
1333804,"""2f72781d-28d9-4bd0-867e-d8a0099fa803""",Curefenac 100mg,9.401714
575320,"""90a24bab-3aed-4383-9509-b0ee96045cd6""",Bunto Blood Caps x100,9.319536


In [17]:
# TOP_K_WHOLE = 100
# RNDCG_MULTIPLIER = 3

## Evaluate
# eval_map_whole = map_at_k(data, all_predictions_whole, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_WHOLE)
# eval_ndcg_whole = ndcg_at_k(data, all_predictions_whole, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_WHOLE)
# eval_rndcg_whole = rndcg_at_k(data, all_predictions_whole, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_WHOLE, rndcg_multiplier=RNDCG_MULTIPLIER)
# eval_precision_whole = precision_at_k(data, all_predictions_whole, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_WHOLE)
# eval_recall_whole = recall_at_k(data, all_predictions_whole, col_user=COL_USER, col_item=COL_ITEM, col_rating=COL_RATING, k=TOP_K_WHOLE)

# print("MAP:\t%f" % eval_map_whole,
#       "NDCG:\t%f" % eval_ndcg_whole,
#       "RNDCG:\t%f" % eval_rndcg_whole,
#       "Precision@K:\t%f" % eval_precision_whole,
#       "Recall@K:\t%f" % eval_recall_whole, sep='\n')

In [18]:
# SAVE_ALL_RECS = True
# SAVE_NEW_RECS = True

## Save recommendations
# if SAVE_ALL_RECS:
#     top_k_predictions_whole = get_top_k_items(all_predictions_whole, col_user=COL_USER, col_rating=DEFAULT_PREDICTION_COL, k=TOP_K_WHOLE)
#     top_k_predictions_whole.drop(DEFAULT_PREDICTION_COL, axis = 1, inplace = True)
#     # add column of true sales and rank of predicted products
#     top_all_true = get_top_k_items(data, col_user=COL_USER, col_rating=COL_RATING, k=data_set.num_items)
#     top_k_predictions_whole = top_k_predictions_whole.merge(top_all_true, how = 'left', on = [COL_USER, COL_ITEM], suffixes = (None, "_true"))
#     top_k_predictions_whole["rank_true"] = top_k_predictions_whole["rank_true"].convert_dtypes()
#     top_k_predictions_whole.rename(columns = {COL_ITEM: "predicted " + COL_ITEM, COL_RATING: "predicted " + COL_ITEM + "'s true " + COL_RATING, "rank_true": "predicted " + COL_ITEM + "'s true rank"}, inplace = True)
#     # add columns of true top-ranked products
#     top_all_true.rename(columns = {COL_ITEM: "true " + COL_ITEM, COL_RATING: "true "  + COL_ITEM + "'s " + COL_RATING}, inplace = True)
#     top_k_predictions_whole = top_k_predictions_whole.merge(top_all_true, how = 'left', on = [COL_USER, 'rank'])
#     # reorder columns
#     top_k_predictions_whole = top_k_predictions_whole[[COL_USER, "rank", "true " + COL_ITEM, "true " + COL_ITEM + "'s " + COL_RATING, "predicted " + COL_ITEM, "predicted "  + COL_ITEM + "'s true " + COL_RATING, "predicted " + COL_ITEM + "'s true rank"]]
#     # save to csv
#     top_k_predictions_whole.to_csv(THIS_ENGINE_NAME + "_" + COUNTRY + "_top_" + str(TOP_K_WHOLE) + "_all_prod_recs.csv")
# if SAVE_NEW_RECS:
#     new_predictions_whole = all_predictions_whole.merge(data, on=[COL_USER,COL_ITEM], indicator=True, how="left").query('_merge=="left_only"').drop('_merge', axis=1).drop([COL_RATING], axis=1)
#     top_k_predictions_whole = get_top_k_items(new_predictions_whole, col_user=COL_USER, col_rating=DEFAULT_PREDICTION_COL, k=TOP_K_WHOLE)
#     top_k_predictions_whole.drop(DEFAULT_PREDICTION_COL, axis = 1, inplace = True)
#     top_k_predictions_whole.rename(columns = {COL_ITEM: "predicted new " + COL_ITEM}, inplace=True)
#     top_k_true = get_top_k_items(data, col_user=COL_USER, col_rating=COL_RATING, k=TOP_K_WHOLE)
#     top_k_true.rename(columns = {COL_ITEM: "true " + COL_ITEM, COL_RATING: "true "  + COL_ITEM + "'s " + COL_RATING}, inplace = True)
#     top_k_predictions_whole = top_k_predictions_whole.merge(top_k_true, on = [COL_USER, 'rank'])
#     top_k_predictions_whole = top_k_predictions_whole[[COL_USER, "rank", "true " + COL_ITEM, "true " + COL_ITEM + "'s " + COL_RATING, "predicted new " + COL_ITEM]]
#     top_k_predictions_whole.to_csv(THIS_ENGINE_NAME + "_" + COUNTRY + "_top_" + str(TOP_K_WHOLE) + "_new_prod_recs.csv")